Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
import datetime

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:    
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
            / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

### Logistic Regression

In [5]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = (tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                labels=tf_train_labels, logits=logits)) + 0.001 * tf.nn.l2_loss(weights))
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 3001
t0 = datetime.datetime.now()

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                  valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
print ('Time: ', datetime.datetime.now() - t0)

Initialized
Minibatch loss at step 0: 19.655592
Minibatch accuracy: 11.7%
Validation accuracy: 18.0%
Minibatch loss at step 500: 3.331386
Minibatch accuracy: 71.9%
Validation accuracy: 76.8%
Minibatch loss at step 1000: 2.094455
Minibatch accuracy: 73.4%
Validation accuracy: 79.2%
Minibatch loss at step 1500: 1.240283
Minibatch accuracy: 82.0%
Validation accuracy: 80.8%
Minibatch loss at step 2000: 1.059801
Minibatch accuracy: 75.8%
Validation accuracy: 80.8%
Minibatch loss at step 2500: 1.080791
Minibatch accuracy: 78.9%
Validation accuracy: 82.5%
Minibatch loss at step 3000: 0.685892
Minibatch accuracy: 84.4%
Validation accuracy: 82.4%
Test accuracy: 87.8%
Time:  0:00:06.209157


### One-layer Neural Network for Logistic Regression

In [7]:
num_steps = 3001
n_hidden_nodes = 1024

def one_layer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_1, weights['out']) + biases['out']
    return out_layer

batch_size = 128

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights = {
        'h1': tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden_nodes])),
        'out': tf.Variable(tf.truncated_normal([n_hidden_nodes, num_labels]))
    }
    biases = {
        'b1': tf.Variable(tf.zeros([n_hidden_nodes])),
        'out': tf.Variable(tf.zeros([num_labels]))
    }

    # Training computation.
    train_logits = one_layer_perceptron(tf_train_dataset, weights, biases)
    valid_logits = one_layer_perceptron(tf_valid_dataset, weights, biases)
    test_logits = one_layer_perceptron(tf_test_dataset, weights, biases)
    loss = (tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                logits=train_logits, labels=tf_train_labels)) 
            + 0.001 * (tf.nn.l2_loss(weights['h1']) + tf.nn.l2_loss(weights['out'])))
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(train_logits)
    valid_prediction = tf.nn.softmax(valid_logits)
    test_prediction = tf.nn.softmax(test_logits)

In [8]:
t0 = datetime.datetime.now()

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                  valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
print ('Time: ', datetime.datetime.now() - t0)

Initialized
Minibatch loss at step 0: 682.505981
Minibatch accuracy: 10.2%
Validation accuracy: 25.0%
Minibatch loss at step 500: 241.977295
Minibatch accuracy: 58.6%
Validation accuracy: 72.2%
Minibatch loss at step 1000: 114.370232
Minibatch accuracy: 78.9%
Validation accuracy: 83.0%
Minibatch loss at step 1500: 68.349831
Minibatch accuracy: 81.2%
Validation accuracy: 83.3%
Minibatch loss at step 2000: 42.352726
Minibatch accuracy: 81.2%
Validation accuracy: 84.4%
Minibatch loss at step 2500: 25.408831
Minibatch accuracy: 82.0%
Validation accuracy: 86.9%
Minibatch loss at step 3000: 15.391706
Minibatch accuracy: 89.1%
Validation accuracy: 87.2%
Test accuracy: 92.6%
Time:  0:01:26.392867


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [61]:
num_steps = 3001
n_hidden_nodes = 1024

def one_layer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_1, weights['out']) + biases['out']
    return out_layer

batch_size = 128

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights = {
        'h1': tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden_nodes])),
        'out': tf.Variable(tf.truncated_normal([n_hidden_nodes, num_labels]))
    }
    biases = {
        'b1': tf.Variable(tf.zeros([n_hidden_nodes])),
        'out': tf.Variable(tf.zeros([num_labels]))
    }

    # Training computation.
    train_logits = one_layer_perceptron(tf_train_dataset, weights, biases)
    valid_logits = one_layer_perceptron(tf_valid_dataset, weights, biases)
    test_logits = one_layer_perceptron(tf_test_dataset, weights, biases)
    loss = (tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                logits=train_logits, labels=tf_train_labels)) 
            + 0.001 * (tf.nn.l2_loss(weights['h1']) + tf.nn.l2_loss(weights['out'])))
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(train_logits)
    valid_prediction = tf.nn.softmax(valid_logits)
    test_prediction = tf.nn.softmax(test_logits)

In [62]:
t0 = datetime.datetime.now()

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels[:128*3].shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                  valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
print ('Time: ', datetime.datetime.now() - t0)

Initialized
Minibatch loss at step 0: 654.324463
Minibatch accuracy: 6.2%
Validation accuracy: 33.4%
Minibatch loss at step 500: 191.035004
Minibatch accuracy: 100.0%
Validation accuracy: 69.2%
Minibatch loss at step 1000: 115.854042
Minibatch accuracy: 100.0%
Validation accuracy: 69.1%
Minibatch loss at step 1500: 70.260323
Minibatch accuracy: 100.0%
Validation accuracy: 69.1%
Minibatch loss at step 2000: 42.609791
Minibatch accuracy: 100.0%
Validation accuracy: 69.1%
Minibatch loss at step 2500: 25.841204
Minibatch accuracy: 100.0%
Validation accuracy: 69.4%
Minibatch loss at step 3000: 15.672304
Minibatch accuracy: 100.0%
Validation accuracy: 70.2%
Test accuracy: 76.2%
Time:  0:01:22.577154


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [71]:
num_steps = 3001
n_hidden_nodes = 1024

def one_layer_perceptron(x, weights, biases, keep_prob=1.0):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    tf_keep_prob = tf.placeholder(tf.float32)
    tf_layer_1_drop = tf.nn.dropout(layer_1, keep_prob)
    # Output layer with linear activation
    out_layer = tf.matmul(tf_layer_1_drop, weights['out']) + biases['out']
    return out_layer

batch_size = 128

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights = {
        'h1': tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden_nodes])),
        'out': tf.Variable(tf.truncated_normal([n_hidden_nodes, num_labels]))
    }
    biases = {
        'b1': tf.Variable(tf.zeros([n_hidden_nodes])),
        'out': tf.Variable(tf.zeros([num_labels]))
    }

    # Training computation.
    train_logits = one_layer_perceptron(tf_train_dataset, weights, biases, keep_prob = 0.8)
    valid_logits = one_layer_perceptron(tf_valid_dataset, weights, biases, keep_prob = 1.0)
    test_logits = one_layer_perceptron(tf_test_dataset, weights, biases, keep_prob = 1.0)
    loss = (tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                logits=train_logits, labels=tf_train_labels)) 
            + 0.001 * (tf.nn.l2_loss(weights['h1']) + tf.nn.l2_loss(weights['out'])))
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(train_logits)
    valid_prediction = tf.nn.softmax(valid_logits)
    test_prediction = tf.nn.softmax(test_logits)

In [72]:
t0 = datetime.datetime.now()

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                  valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
print ('Time: ', datetime.datetime.now() - t0)

Initialized
Minibatch loss at step 0: 721.452271
Minibatch accuracy: 10.9%
Validation accuracy: 25.4%
Minibatch loss at step 500: 208.515839
Minibatch accuracy: 66.4%
Validation accuracy: 78.0%
Minibatch loss at step 1000: 115.228088
Minibatch accuracy: 76.6%
Validation accuracy: 82.4%
Minibatch loss at step 1500: 68.765511
Minibatch accuracy: 81.2%
Validation accuracy: 84.0%
Minibatch loss at step 2000: 42.702564
Minibatch accuracy: 80.5%
Validation accuracy: 84.4%
Minibatch loss at step 2500: 25.363422
Minibatch accuracy: 80.5%
Validation accuracy: 86.6%
Minibatch loss at step 3000: 15.345823
Minibatch accuracy: 92.2%
Validation accuracy: 87.1%
Test accuracy: 92.4%
Time:  0:01:38.593042


### Overfitting case

In [73]:
num_steps = 3001
n_hidden_nodes = 1024

def one_layer_perceptron(x, weights, biases, keep_prob=1.0):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    tf_keep_prob = tf.placeholder(tf.float32)
    tf_layer_1_drop = tf.nn.dropout(layer_1, keep_prob)
    # Output layer with linear activation
    out_layer = tf.matmul(tf_layer_1_drop, weights['out']) + biases['out']
    return out_layer

batch_size = 128

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights = {
        'h1': tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden_nodes])),
        'out': tf.Variable(tf.truncated_normal([n_hidden_nodes, num_labels]))
    }
    biases = {
        'b1': tf.Variable(tf.zeros([n_hidden_nodes])),
        'out': tf.Variable(tf.zeros([num_labels]))
    }

    # Training computation.
    train_logits = one_layer_perceptron(tf_train_dataset, weights, biases, keep_prob=0.5)
    valid_logits = one_layer_perceptron(tf_valid_dataset, weights, biases, keep_prob=1.)
    test_logits = one_layer_perceptron(tf_test_dataset, weights, biases, keep_prob=1.)
    loss = (tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                logits=train_logits, labels=tf_train_labels)) 
            + 0.001 * (tf.nn.l2_loss(weights['h1']) + tf.nn.l2_loss(weights['out'])))
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(train_logits)
    valid_prediction = tf.nn.softmax(valid_logits)
    test_prediction = tf.nn.softmax(test_logits)

In [74]:
t0 = datetime.datetime.now()

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels[:128*3].shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                  valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
print ('Time: ', datetime.datetime.now() - t0)

Initialized
Minibatch loss at step 0: 765.935059
Minibatch accuracy: 7.8%
Validation accuracy: 36.9%
Minibatch loss at step 500: 191.981995
Minibatch accuracy: 100.0%
Validation accuracy: 75.9%
Minibatch loss at step 1000: 116.470123
Minibatch accuracy: 100.0%
Validation accuracy: 75.3%
Minibatch loss at step 1500: 70.643204
Minibatch accuracy: 100.0%
Validation accuracy: 75.9%
Minibatch loss at step 2000: 42.843784
Minibatch accuracy: 100.0%
Validation accuracy: 75.8%
Minibatch loss at step 2500: 25.984385
Minibatch accuracy: 100.0%
Validation accuracy: 75.9%
Minibatch loss at step 3000: 15.759253
Minibatch accuracy: 100.0%
Validation accuracy: 75.8%
Test accuracy: 82.0%
Time:  0:01:39.932983


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [27]:
num_steps = 100001
n_hidden_nodes = 64

def multi_layer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.add(tf.matmul(layer_2, weights['out']), biases['out'])
    return out_layer

batch_size = 128

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights = {
        'h1': tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden_nodes])),
        'h2': tf.Variable(tf.truncated_normal([n_hidden_nodes, n_hidden_nodes])),
        'out': tf.Variable(tf.truncated_normal([n_hidden_nodes, num_labels]))
    }
    biases = {
        'b1': tf.Variable(tf.zeros([n_hidden_nodes])),
        'b2': tf.Variable(tf.zeros([n_hidden_nodes])),
        'out': tf.Variable(tf.zeros([num_labels]))
    }

    # Training computation.
    train_logits = multi_layer_perceptron(tf_train_dataset, weights, biases)
    valid_logits = multi_layer_perceptron(tf_valid_dataset, weights, biases)
    test_logits = multi_layer_perceptron(tf_test_dataset, weights, biases)
    loss = (tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                logits=train_logits, labels=tf_train_labels)) 
            + 0.001 * (tf.nn.l2_loss(weights['h1']) + 
                       tf.nn.l2_loss(weights['h2']) + 
                       tf.nn.l2_loss(weights['out'])))
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(train_logits)
    valid_prediction = tf.nn.softmax(valid_logits)
    test_prediction = tf.nn.softmax(test_logits)

In [28]:
t0 = datetime.datetime.now()

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 10000 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                  valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
print ('Time: ', datetime.datetime.now() - t0)

Initialized
Minibatch loss at step 0: 387.420471
Minibatch accuracy: 4.7%
Validation accuracy: 17.1%
Minibatch loss at step 10000: 3.415878
Minibatch accuracy: 78.9%
Validation accuracy: 83.5%
Minibatch loss at step 20000: 0.889154
Minibatch accuracy: 85.2%
Validation accuracy: 86.5%
Minibatch loss at step 30000: 0.326970
Minibatch accuracy: 93.8%
Validation accuracy: 88.4%
Minibatch loss at step 40000: 0.289380
Minibatch accuracy: 93.8%
Validation accuracy: 89.0%
Minibatch loss at step 50000: 0.403823
Minibatch accuracy: 89.8%
Validation accuracy: 89.2%
Minibatch loss at step 60000: 0.391106
Minibatch accuracy: 91.4%
Validation accuracy: 89.1%
Minibatch loss at step 70000: 0.370575
Minibatch accuracy: 91.4%
Validation accuracy: 89.4%
Minibatch loss at step 80000: 0.431421
Minibatch accuracy: 86.7%
Validation accuracy: 89.4%
Minibatch loss at step 90000: 0.400249
Minibatch accuracy: 91.4%
Validation accuracy: 89.5%
Minibatch loss at step 100000: 0.458965
Minibatch accuracy: 88.3%
Valid